In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
cats = requests.get('https://api.mercadolibre.com/sites/MLA/categories')
cat = cats.json()
categorias=[cat[i]['id'] for i in range(len(cat))]
offset = [str(i) for i in range(0, 951, 50)]

In [62]:
product_id=[]
precios=[]
seller = []
categorias_list=[]
for categoria in categorias:
  for i in offset:
    url = f'https://api.mercadolibre.com/sites/MLA/search?category={categoria}&offset={i}'
    request = requests.get(url)
    items = request.json()
    for e in range(len(items['results'])):
        precios.append(items['results'][e]['price'])
        product_id.append(items['results'][e]['catalog_product_id'])
        seller.append(items['results'][e]['seller']['nickname'])
        categorias_list.append(categoria)
diccionario = {
    'productos_id': product_id,
    'precios': precios,
    'vendedor': seller,
    'categorias': categorias_list
}

df = pd.DataFrame(diccionario)

In [81]:
df = pd.DataFrame(diccionario)
df = df.drop_duplicates()
df_agrupado = df.groupby('productos_id')['precios'].mean().reset_index()
df_agrupado.rename(columns={'precios': 'media_precio'}, inplace=True)
df = pd.merge(df, df_agrupado, on='productos_id')
df['precio_vs_media'] = df['precios'] - df['media_precio']
df['descuento'] = np.where(df['precio_vs_media']>0, df['precio_vs_media']/df['precios']*100, 0)
df_descuentos = df[df['descuento'] > 5]

In [ ]:
for categoria in categorias:
  df_cat= df_descuentos[df_descuentos['categorias'] == categoria]
  plt.figure(figsize=(12, 6))
  plt.bar(df_cat['productos_id'], df_cat['descuento'], color='skyblue')
  plt.title('Descuento por Producto_ID en la categoría ' +categoria )
  plt.xlabel('Producto_ID')
  plt.ylabel('Descuento')
  plt.xticks(rotation=45)
  plt.grid(axis='y', linestyle='--', linewidth=0.5)
  plt.tight_layout()
  plt.show()

In [ ]:
encontrar_repetidos(df_descuentos['productos_id'])

In [85]:
df_descuentos[df_descuentos['productos_id']=='MLA22069718']

,productos_id,precios,vendedor,categorias,media_precio,precio_vs_media,descuento
9671,MLA22069718,42188.18,PCMACCESORIOS,MLA1648,37576.453333,4611.726667,10.931324
9672,MLA22069718,42188.18,PCMACCESORIOS,MLA1144,37576.453333,4611.726667,10.931324


In [ ]:

plt.figure(figsize=(10, 5))
df_descuentos.groupby('categorias')['productos_id'].count().plot(kind='bar', color='salmon')
plt.xlabel('Categoría')
plt.ylabel('Recuento de Productos para hacer descuento')
plt.title('Recuento de Productos por Categoría')
plt.xticks(rotation=45, ha='right')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()


In [ ]:
df_descuentos.groupby('categorias').size().reset_index(name='cantidad_con_descuento')

In [ ]:
df_descuentos.groupby('categorias')['descuento'].mean().reset_index(name='media_descuento')